In [7]:
# need to know which ones are ensemble:
from glob import glob
import os

import os
import sys
from paths import libpath
sys.path.append(libpath)
from utilsdf import list_files_by_tilenames,tile_files_to_parquet_parallel
from uvars import (tilenames_mkd, tilenames_tls,tilenames_rgn,
                   tilenames_lidar,RES_DPATH)

from uvars import aux_ending12,s1_ending12,s2_ending12,tar_ending12

In [22]:
import os
import pandas as pd
from catboost import CatBoostRegressor, Pool
from pprint import pprint

def load_cb_model(model_path):
    """
    Loads a CatBoost model from the given path.
    
    Parameters:
        model_path (str): Path to the CatBoost model file.
    
    Returns:
        CatBoostRegressor: Loaded CatBoost model.
    """
    assert os.path.exists(model_path), f"{model_path} does not exist"
    model = CatBoostRegressor()
    model.load_model(model_path)
    return model

def predict_with_models(model_dir, df, fcol, yvar, tcol):
    """
    Predicts using all CatBoost models in the specified directory, averages the predictions,
    and calculates residuals. Returns the updated dataframe.

    Parameters:
        model_dir (str): Path to the directory containing the model files.
        df (pd.DataFrame): Dataframe containing the features and true values.
        fcol (list): List of feature column names to use as predictors.
        yvar (str): Target variable name for predictions.
        tcol (str): Column name of the true target values to calculate residuals.

    Returns:
        pd.DataFrame: Updated dataframe with predictions and residuals.
    """
    # Get all model files ending with "_model.txt"
    model_files = [os.path.join(model_dir, f) for f in os.listdir(model_dir) if f.endswith("_model.txt")]
    print('#--------------------------------------------------#')
    pprint(model_files)
    assert model_files, f"No model files found in {model_dir}"

    # Create a CatBoost Pool for the input data
    data_pool = Pool(df[fcol])

    # Load models and make predictions
    predictions = []
    for model_file in model_files:
        print(f"Loading model: {model_file}")
        model = load_cb_model(model_file)
        y_pred = model.predict(data_pool)
        predictions.append(y_pred)

    # Calculate the average predictions
    y_pred_avg = sum(predictions) / len(predictions)

    # Add predictions and residuals to the dataframe
    df[f'ml_{yvar}'] = y_pred_avg
    df[f'ml_{tcol}'] = df[tcol].subtract(df[f'ml_{yvar}'])

    return df

# Example Usage:
# Assuming `df` is your dataframe, `model_dir` is the folder with models, and `fcol` contains feature columns:
# model_dir = "path/to/models"
# df = predict_with_models(model_dir, df, fcol=['feature1', 'feature2'], yvar='target', tcol='actual_target')


In [24]:
import rasterio
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from utilsdf import check_fillnulls

def get_parquets_and_geotifs_by_tile(RES_DPATH, X, tilenames,vending_all):
    fparquet_list,tile_files_list = list_files_by_tilenames(RES_DPATH, X, tilenames)
    assert len(fparquet_list) == len(tile_files_list), 'len(fparquet_list) != len(tile_files_list)'
    _, fparquet_list = tile_files_to_parquet_parallel(tilenames, 
                                                       RES_DPATH, 
                                                       X, 
                                                       vending_all)
    
    return fparquet_list,tile_files_list   


def write_predictions(predictions, tile_file, output_raster_path, block_size=(128, 128)):
    """
    Writes predictions to a new raster file using metadata from an existing tile file in blocks,
    optimised for large rasters.

    Parameters:
    - predictions (array-like): 1D array of predicted values matching the flattened raster size.
    - tile_file (str): Path to the raster file from which metadata will be read.
    - output_raster_path (str): Path where the new raster file will be saved.
    - block_size (tuple): Tuple specifying the block size for processing (default is (128, 128)).

    Returns:
    - None
    """
    # Read metadata and raster dimensions from the tile file
    with rasterio.open(tile_file) as src:
        meta = src.meta.copy()
        raster_shape = (src.height, src.width)
        transform = src.transform
        crs = src.crs

    # Reshape predictions to match the raster's dimensions
    try:
        predictions_reshaped = np.array(predictions).reshape(raster_shape)
    except ValueError:
        raise ValueError(f"Predictions array size {len(predictions)} does not match raster dimensions {raster_shape}.")

    # Update metadata for writing a new raster
    meta.update({
        "dtype": rasterio.float32,  # Ensure predictions are stored as float32
        "count": 1,  # Single band
        "compress": "lzw"  # Optional: Add compression
    })

    # Write the new raster in blocks
    with rasterio.open(output_raster_path, "w", **meta) as dst:
        print(f"Writing raster in blocks of size: {block_size}")
        for y_start in range(0, raster_shape[0], block_size[0]):
            for x_start in range(0, raster_shape[1], block_size[1]):
                # Calculate window bounds
                y_end = min(y_start + block_size[0], raster_shape[0])
                x_end = min(x_start + block_size[1], raster_shape[1])
                window = ((y_start, y_end), (x_start, x_end))

                # Slice the data for the current block
                block_data = predictions_reshaped[y_start:y_end, x_start:x_end]

                # Write the block to the corresponding window
                dst.write(block_data.astype(rasterio.float32), 1, window=rasterio.windows.Window.from_slices(*window))

    print(f"Raster written successfully to {output_raster_path}")



def get_tile_file(tile_files):
    tile_file = [i for i in tile_files if i.endswith('edem_W84.tif')] #[0]
    assert len(tile_file) == 1, 'len(tile_file) != 1'
    tile_file = tile_file[0]
    return tile_file

def load_prediction_data(fparquet,fcol):
    df = pd.read_parquet(fparquet)
    df[fcol] = check_fillnulls(df[fcol])
    return df

def load_cb_model(modelpath):
    assert os.path.exists(modelpath), f'{modelpath} does not exist'
    model = CatBoostRegressor()
    model.load_model(modelpath)
    return model

def get_prediction_df(model,df,fcol,yvar,tcol):
    df[f'ml_{yvar}'] = model.predict(Pool(df[fcol]))
    df[f'ml_{tcol}'] = df[tcol].subtract(df[f'ml_{yvar}'])
    return df

def cb_predict_workflow(outdir,modelpath,fparquet,tile_files,
                        fcol,yvar,tcol,ps,bsize=256):
    tile_ifile = get_tile_file(tile_files)
    tile_odir = os.path.join(outdir,tile_ifile.split('/')[-2])
    os.makedirs(tile_odir,exist_ok=True)
    tile_ofile = os.path.join(tile_odir,tile_ifile.split('/')[-1].replace('.tif','_ML.tif'))

    if not os.path.isfile(tile_ofile):
        f'writing:: {tile_ofile} ...'
        df = load_prediction_data(fparquet,fcol)
        assert len(df) == ps * ps, 'Grid size does not match'
        model = load_cb_model(modelpath)
        df = get_prediction_df(model,df,fcol,yvar,tcol)

        
        write_predictions(predictions=df[f'ml_{tcol}'], #
                        tile_file=tile_ifile, 
                        output_raster_path=tile_ofile, 
                        block_size=(bsize, bsize))
    else:
        print(f'{tile_ofile} already exists')


yvar = "zdif"
tcol = 'edem_w84'
rcol = 'multi_dtm_lidar'
fcol = ['egm08', 'egm96', 'tdem_hem', 'multi_s1_band1', 'multi_s1_band2',
        'multi_s2_band1', 'multi_s2_band2', 'multi_s2_band3']
tar_ending,aux_ending,s1_ending,s2_ending = aux_ending12,s1_ending12,s2_ending12,tar_ending12
vending_all = tar_ending+aux_ending+s1_ending+s2_ending
X = 12 
ps = 9001
tilenames = tilenames_mkd
bsize = 512 # match with grid size X 
modelpath = ''
outdir = ''
#fparquet_list,tile_files_list = get_parquets_and_geotifs_by_tile(RES_DPATH, X, tilenames_mkd,vending_all)

# for fparquet,tile_files in zip(fparquet_list,tile_files_list):
#     cb_predict_workflow(outdir,modelpath,fparquet,tile_files,
#                         fcol,yvar,tcol,ps,bsize=256)
    
## clean this code, and make it more modular and improve any thing that needs improvement
## add timer per cb_predict_workflow 
# add timer for the whole process too see how long it takes to predict all the tiles
# print the timer at the end and write both times to the log file txt t

In [25]:
#model_dir = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/MODELS/train_cb_bysample/12/zdif/iter10000_n236435487_eqallxtile/"

yvar = "zdif"
tcol = 'edem_w84'
rcol = 'multi_dtm_lidar'
fcol = ['egm08', 'egm96', 'tdem_hem', 'multi_s1_band1', 'multi_s1_band2',
        'multi_s2_band1', 'multi_s2_band2', 'multi_s2_band3']
tar_ending,aux_ending,s1_ending,s2_ending = aux_ending12,s1_ending12,s2_ending12,tar_ending12
vending_all = tar_ending+aux_ending+s1_ending+s2_ending
X = 12 
ps = 9001
tilenames = tilenames_mkd
bsize = 512 # match with grid size X 
# modelpath = ''
# outdir = ''
# fparquet_list,tile_files_list = get_parquets_and_geotifs_by_tile(RES_DPATH, X, tilenames_mkd,vending_all)


In [26]:
def cbe_predict_workflow(outdir,model_dir,dirname,fparquet,tile_files,
                        fcol,yvar,tcol,ps,bsize=256):
    # improve this so that it checks models before loading the data 
    tile_ifile = get_tile_file(tile_files)
    tile_odir = os.path.join(outdir,dirname,tile_ifile.split('/')[-2])
    os.makedirs(tile_odir,exist_ok=True)
    tile_ofile = os.path.join(tile_odir,tile_ifile.split('/')[-1].replace('.tif','_ML.tif'))

    if not os.path.isfile(tile_ofile):
        f'writing:: {tile_ofile} ...'
        df = load_prediction_data(fparquet,fcol)
        assert len(df) == ps * ps, 'Grid size does not match'
        #model = load_cb_model(modelpath)
        #df = get_prediction_df(model,df,fcol,yvar,tcol)
        df = predict_with_models(model_dir, df, fcol, yvar, tcol)

        
        write_predictions(predictions=df[f'ml_{tcol}'], #
                        tile_file=tile_ifile, 
                        output_raster_path=tile_ofile, 
                        block_size=(bsize, bsize))
    else:
        print(f'{tile_ofile} already exists')

In [29]:
fparquet_list, tile_files_list = get_parquets_and_geotifs_by_tile(RES_DPATH, X, tilenames, vending_all)

Filtered files count: 8/25
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N09E105/N09E105_byldem.parquet
Filtered files count: 8/25
Filtered files count: 8/25
Filtered files count: 8/25
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N10E105/N10E105_byldem.parquet
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N09E106/N09E106_byldem.parquet
Filtered files count: 8/25
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N10E104/N10E104_byldem.parquet
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N10E106/N10E106_byldem.parquet


In [28]:
dirname = "iter5000_n236435487_eqallxtile"#"iter10000_n236435487_eqallxtile"
outdir = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/PREDICTIONS/TESTING"
model_dir = f"/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/MODELS/train_cb_bysample/12/zdif/{dirname}/"
fparquet_list, tile_files_list = get_parquets_and_geotifs_by_tile(RES_DPATH, X, tilenames, vending_all)
#fparquet, tile_files  = fparquet_list[0], tile_files_list[0]
for fparquet,tile_files in zip(fparquet_list,tile_files_list):
    cbe_predict_workflow(outdir,model_dir,dirname,fparquet,tile_files,
                        fcol,yvar,tcol,ps,bsize=256)

Filtered files count: 8/25
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N09E105/N09E105_byldem.parquet
Filtered files count: 8/25
Filtered files count: 8/25
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N10E104/N10E104_byldem.parquet
Filtered files count: 8/25
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N09E106/N09E106_byldem.parquet
Filtered files count: 8/25
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N10E105/N10E105_byldem.parquet
Parquet file already exists: /media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N10E106/N10E106_byldem.parquet
#--------------------------------------------------#
['/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/MODELS/train_cb_bysample/12/zdif/iter5000_n236435487_eqallxtile/catboost_5000_13_model.txt',
 '/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/MODELS/train_cb_bysample/12/zdif/iter5000_n236435487_eqallxtile/catboost_5

/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/PREDICTIONS/TESTING/iter10000_n236435487_eqallxtile/N09E105/N09E105_edem_W84_ML.tif already exists


In [5]:
cb_emodel_paths = glob(f'{cb_pathe}*.txt')

In [8]:
os.listdir(cb_pathe)

['catboost_10000_13_metrics.csv',
 'catboost_10000_13_model.txt',
 'catboost_10000_21_metrics.csv',
 'catboost_10000_21_model.txt',
 'catboost_10000_43_metrics.csv',
 'catboost_10000_43_model.txt']

In [ ]:
# go throuth the folder, and select all the txt files that are the model
['catboost_10000_13_metrics.csv',
 'catboost_10000_13_model.txt',
 'catboost_10000_21_metrics.csv',
 'catboost_10000_21_model.txt',
 'catboost_10000_43_metrics.csv',
 'catboost_10000_43_model.txt']

def load_cb_model(modelpath):
    assert os.path.exists(modelpath), f'{modelpath} does not exist'
    model = CatBoostRegressor()
    model.load_model(modelpath)
    return model

# modify the function above to make predciton to all models and take the avarage 
use this as x data Pool(df[fcol])
save y_pred_avg  
and then add to df as df[f'ml_{yvar}'] = y_pred_avg
df[f'ml_{tcol}'] = df[tcol].subtract(df[f'ml_{yvar}'])
and return as df in the final fnction 

# write high perfomannt function, andmake it modular 



In [6]:
cb_emodel_paths

['/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/MODELS/train_cb_bysample/12/zdif/iter10000_n236435487_eqallxtile/catboost_10000_13_model.txt',
 '/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/MODELS/train_cb_bysample/12/zdif/iter10000_n236435487_eqallxtile/catboost_10000_21_model.txt',
 '/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/MODELS/train_cb_bysample/12/zdif/iter10000_n236435487_eqallxtile/catboost_10000_43_model.txt']